In [ ]:
from dataloader import ccd_load_data, transform
from train_val_loop import training
from DSTA import DSTA
from TSAA_Module import SelfAttentionAggregation
from exp_loss_function import exp_loss
import torch.nn as nn
import torch
from feature_extraction import init_feature_extractor
from ultralytics import YOLO

In [ ]:
#!mkdir "/content/CarCrach"
#!mkdir "/content/CarCrach/Crash"
#!mkdir "/content/CarCrach/Normal"

In [ ]:
#!unzip "/content/drive/MyDrive/CarCrash/videos/Crash-1500.zip" -d "/content/CarCrach/Crash"
#!unzip "/content/drive/MyDrive/CarCrash/videos/Normal.zip" -d "/content/CarCrach/Normal"
#!cp "/content/drive/MyDrive/CarCrash/videos/Crash-1500.txt" "/content/"

In [ ]:
#!pip install ultralytics

In [ ]:
data_path = "/content/CarCrach"
annote_path = "/content/Crash-1500.txt"

In [ ]:
data_loader = ccd_load_data (data_path = data_path, annote_path = annote_path, transform=transform, batch_size=1, shuffle=True)

In [ ]:
feature_extractor = init_feature_extractor(backbone='vgg16', device=torch.device('cuda'))
object_detector = YOLO('yolov8m.pt')
device = torch.device('cuda')
dsta = DSTA(device=device, d=4096, m=8, input_dim=4096+4096, output_dim=2, n_layers=1, n_objects=5, object_detector=object_detector, feature_extractor=feature_extractor)
tsaa = SelfAttentionAggregation(device=device, t=50, d=4096)
optimizer_dsta = torch.optim.Adam(params=dsta.parameters(), lr=0.01)
optimizer_tsaa = torch.optim.Adam(params=tsaa.parameters(), lr=0.01)
num_epochs = 10

In [ ]:
for name, param in dsta.named_parameters():
    print(name)

In [ ]:
training(num_epochs=num_epochs,
         dsta=dsta,
         tsaa=tsaa,
         optimizer_dsta=optimizer_dsta,
         optimizer_tsaa=optimizer_tsaa,
         data_loader=data_loader,
         device=device,
         update_grad_on_all_data=False)